### 概念

* 多输入通道：需要使用三维核矩阵，其中各维度含义为(输入通道数,高,宽)
* 多输出通道：需要使用四维核矩阵，其中各维度含义为(输出通道数,输入通道数,高,宽)
* $1\times1$卷积层
    * 不改变空间维度，只在通道数层面做融合，相当于一个全连接
    * 常用于调整网络层的通道数量和控制模型复杂性

### 代码实现

In [2]:
import torch
from d2l import torch as d2l

In [3]:
# 定义多输入通道互相关运算
def corr2d_multi_in(X, K):
    # 先遍历X和K的第0个维度(通道维度),再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))


X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [4]:
# 定义多输出通道互相关运算
def corr2d_multi_in_out(X, K):
    # 迭代K的第0个维度,每次都对输入X执行互相关运算
    # 最后将所有结果都叠加在一起(由stack函数负责)
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)


K = torch.stack((K, K + 1, K + 2), 0)
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [7]:
# 定义1x1卷积操作:利用全连接实现
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))


X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
corr2d_multi_in_out_1x1(X, K)

tensor([[[-0.7326, -0.4168, -0.2784],
         [ 0.6417,  0.0029,  0.1100],
         [ 1.8982,  0.0622,  0.1787]],

        [[ 0.6827, -0.1933, -0.3964],
         [-0.7090,  0.0324, -0.4051],
         [-0.0790, -0.4383,  0.3102]]])